# AI Solutions Engineer Interview Assignment – Palindrome Data

This notebook implements an end‑to‑end analysis of synthetic WhatsApp‑style conversations between an AI health chatbot and clients. The goals are:

1. **Ingest and parse** the conversation dataset.
2. **Generate risk scores** for:
   - HIV acquisition risk
   - Mental health disorder risk
3. **Produce a structured recommendation and care plan** aligned with high‑level principles from South African National Department of Health (NDOH) guidance.

> **Important note**  
> This notebook is for a technical interview task only. All risk scores and care plans here are simple rule‑based prototypes, **not clinical tools** and **not medical advice**. Any real‑world deployment would need expert clinical review, validation, and strict safety governance.

In [ ]:
import re
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple

import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 200)

DATA_PATH = Path('health_ai_whatsapp_100_conversations_long.txt')
assert DATA_PATH.exists(), f"Dataset not found at {DATA_PATH.resolve()}"

## 1. Data ingestion and parsing

The raw file contains multiple conversations separated by a line:

```text
========== Conversation ==========
```

Within each conversation, each message has the format:

```text
[01/01/2025, 08:00] User: Hi, I need help about something sensitive.
```

We will parse this into a tidy dataframe with columns:
`conversation_id`, `timestamp_str`, `speaker`, `text`, `turn_index`.

In [ ]:
raw_text = DATA_PATH.read_text(encoding='utf-8')

CONV_SEP = '========== Conversation =========='
raw_conversations = [c.strip() for c in raw_text.split(CONV_SEP) if c.strip()]
len(raw_conversations), raw_conversations[0][:400]

In [ ]:
pattern_str = r'^\[(?P<timestamp>[^\]]+)\]\s*(?P<speaker>[^:]+):\s*(?P<text>.*)$'
message_pattern = re.compile(pattern_str, re.MULTILINE)

records = []
for conv_id, conv in enumerate(raw_conversations, start=1):
    for turn_idx, match in enumerate(message_pattern.finditer(conv), start=1):
        records.append(
            {
                'conversation_id': conv_id,
                'turn_index': turn_idx,
                'timestamp_str': match.group('timestamp'),
                'speaker': match.group('speaker').strip(),
                'text': match.group('text').strip(),
            }
        )

df_msgs = pd.DataFrame(records)
df_msgs.head()

In [ ]:
df_msgs['speaker'].value_counts()

## 2. Conversation‑level aggregation

Risk scoring will happen at **conversation level**, based on all user messages in that chat.

In [ ]:
def aggregate_conversation(df: pd.DataFrame) -> pd.DataFrame:
    # Concatenate all user utterances per conversation
    user_text = (
        df[df['speaker'].str.lower().str.contains('user')]
        .groupby('conversation_id')['text']
        .apply(lambda s: '\n'.join(s))
    )
    # Concatenate all AI utterances per conversation (for context / later analysis)
    ai_text = (
        df[df['speaker'].str.lower().str.contains('ai')]
        .groupby('conversation_id')['text']
        .apply(lambda s: '\n'.join(s))
    )

    agg = (
        pd.DataFrame({'user_text': user_text})
        .join(pd.DataFrame({'ai_text': ai_text}))
        .reset_index()
    )
    return agg

df_conv = aggregate_conversation(df_msgs)
df_conv.head()

## 3. Simple lexicon‑based risk model

Given we do not have labelled data, we use a **transparent rule‑based approach**:

### HIV acquisition risk

*High‑risk indicators* (e.g. unprotected sex, known HIV‑positive partner, multiple partners, needle sharing, STI symptoms) are assigned higher weights.

### Mental‑health risk

*Crisis indicators* (e.g. suicidal thoughts, self‑harm) and *moderate indicators* (e.g. persistent low mood, anxiety, insomnia) are scored separately.

Scores are normalised to the **0–1** range and mapped to categorical risk levels.

In [ ]:
# Keyword lexicons – deliberately simple and interpretable

HIV_HIGH_KEYWORDS = [
    'unprotected sex', 'no condom', 'without a condom', 'condom broke',
    'multiple partners', 'many partners', 'sex worker', 'paid sex',
    'needle', 'injecting drugs', 'shared syringe',
    'hiv positive', 'partner is positive', 'sti', 'std', 'genital sore', 'ulcer',
]

HIV_MODERATE_KEYWORDS = [
    'new partner', 'one night stand', 'hookup', 'blood contact',
    'condom slipped', 'missed prep', 'missed pep',
]

MH_CRISIS_KEYWORDS = [
    'suicide', 'suicidal', 'kill myself', 'end my life', 'want to die',
    'self-harm', 'self harm', 'cut myself', 'overdose',
]

MH_MODERATE_KEYWORDS = [
    'depressed', 'depression', 'anxious', 'anxiety', 'panic attack',
    'no energy', "can't sleep", 'insomnia', 'worthless', 'hopeless',
    'stressed', 'stress', 'burnt out', 'burnout',
]

def count_keyword_hits(text: str, keywords: List[str]) -> int:
    if not isinstance(text, str):
        return 0
    text_lower = text.lower()
    return sum(text_lower.count(k) for k in keywords)

def normalise_score(raw: float, max_ref: float) -> float:
    if max_ref <= 0:
        return 0.0
    return min(raw / max_ref, 1.0)

def risk_level(score: float, low=0.25, high=0.6) -> str:
    if score >= high:
        return 'high'
    elif score >= low:
        return 'moderate'
    return 'low'

In [ ]:
def score_conversation(text: str) -> Dict[str, float]:
    """Return raw and normalised scores for HIV and mental‑health risk."""
    hiv_raw = (
        2 * count_keyword_hits(text, HIV_HIGH_KEYWORDS)
        + 1 * count_keyword_hits(text, HIV_MODERATE_KEYWORDS)
    )
    mh_raw = (
        3 * count_keyword_hits(text, MH_CRISIS_KEYWORDS)
        + 1 * count_keyword_hits(text, MH_MODERATE_KEYWORDS)
    )

    hiv_score = normalise_score(hiv_raw, max_ref=6)  # heuristics
    mh_score = normalise_score(mh_raw, max_ref=6)

    return {
        'hiv_raw': hiv_raw,
        'mh_raw': mh_raw,
        'hiv_score': hiv_score,
        'mh_score': mh_score,
        'hiv_level': risk_level(hiv_score),
        'mh_level': risk_level(mh_score),
    }

# Test on a synthetic example
example_text = "I had unprotected sex with a new partner and I am very anxious and can't sleep."
score_conversation(example_text)

## 4. Apply risk scoring to all conversations

In [ ]:
scores = df_conv['user_text'].apply(score_conversation).apply(pd.Series)
df_scored = pd.concat([df_conv[['conversation_id', 'user_text']], scores], axis=1)
df_scored.head()

In [ ]:
df_scored[['hiv_level', 'mh_level']].value_counts()

## 5. Recommendation and care‑plan generator

Here we generate **non‑prescriptive**, high‑level guidance that *resembles* what a South African NDOH‑aligned triage system might propose. The aim is to show how an AI system can structure recommendations – **not** to replace clinicians.

We:
- Combine HIV and mental‑health risk levels.
- Produce a short summary of concerns.
- Suggest action steps: e.g. HIV testing, linkage to care, psycho‑social support, emergency referral if crisis keywords are detected.

In [ ]:
def generate_recommendation(row: pd.Series) -> Dict[str, str]:
    hiv_level = row['hiv_level']
    mh_level = row['mh_level']
    text = row['user_text'].lower()

    summary_parts = []
    if hiv_level != 'low':
        summary_parts.append(f'HIV exposure risk appears **{hiv_level}** based on reported behaviour.')
    else:
        summary_parts.append('No explicit high‑risk sexual or blood‑exposure behaviour detected in the text; HIV risk appears **low** from conversation alone.')

    if mh_level != 'low':
        summary_parts.append(f'Mental‑health concern appears **{mh_level}** based on language suggesting distress.')
    else:
        summary_parts.append('No strong crisis language detected; mental‑health risk appears **low**, although stress is still possible.')

    summary = ' '.join(summary_parts)

    steps = []
    # HIV‑related suggested actions (non‑clinical)
    if hiv_level == 'high':
        steps.append(
            '- Offer **immediate HIV counselling and testing** at the nearest clinic and discuss options such as PEP/PrEP in line with local guidelines.'
        )
    elif hiv_level == 'moderate':
        steps.append(
            '- Encourage the client to attend an HIV testing service within the next few days and provide information on safer‑sex practices.'
        )
    else:
        steps.append(
            '- Provide general HIV prevention education (condom use, testing at routine intervals) and share information on local testing sites.'
        )

    # Mental‑health‑related suggested actions (non‑clinical)
    crisis_present = any(kw in text for kw in MH_CRISIS_KEYWORDS)
    if crisis_present:
        steps.append(
            '- Treat as **potential emergency**: advise immediate contact with local emergency services or same‑day in‑person mental‑health assessment; ensure the person is not left alone.'
        )
    elif mh_level == 'high':
        steps.append(
            '- Arrange a **priority referral** for a mental‑health assessment (e.g. psychologist/psychiatric nurse) and discuss safety planning.'
        )
    elif mh_level == 'moderate':
        steps.append(
            '- Offer counselling support, psycho‑education on stress, sleep, and coping skills; schedule follow‑up contact to monitor symptoms.'
        )
    else:
        steps.append(
            '- Offer brief reassurance, normalise seeking help early, and provide resources for support if symptoms worsen.'
        )

    plan_text = '\n'.join(steps)
    return {
        'summary': summary,
        'care_plan': plan_text,
    }

recs = df_scored.apply(generate_recommendation, axis=1).apply(pd.Series)
df_final = pd.concat([df_scored, recs], axis=1)
df_final.head()

## 6. Inspect a single case end‑to‑end

Below we:
1. Select a conversation by ID.
2. Show the raw turns.
3. Display the computed risk scores and generated care plan.

In [ ]:
def show_conversation(conversation_id: int):
    display(df_msgs[df_msgs['conversation_id'] == conversation_id])
    display(
        df_final[df_final['conversation_id'] == conversation_id][[
            'hiv_score', 'hiv_level', 'mh_score', 'mh_level', 'summary', 'care_plan'
        ]]
    )

# Example: inspect conversation 1
show_conversation(1)

## 7. Export conversation‑level outputs

For convenience, we export the per‑conversation results to a CSV file. This could be used by downstream dashboards or reporting tools.

In [ ]:
from pathlib import Path
OUTPUT_PATH = Path('conversation_risk_scores_and_plans.csv')
df_final.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH.resolve()

## 8. Limitations and next steps

- This is a **rule‑based, lexicon‑driven prototype** – easy to explain but not calibrated.
- Real‑world deployment would require:
  - Larger, labelled datasets and supervised models (e.g. fine‑tuned transformers).
  - Rigorous evaluation (sensitivity/specificity, fairness, and calibration). 
  - Alignment with up‑to‑date NDOH clinical guidelines, reviewed by domain experts.
  - Strong safety controls (escalation flows, human‑in‑the‑loop, audit logging).
- Future iterations could use:
  - Sentence‑level classifiers for specific intents (e.g. *condom failure*, *suicidal ideation*).
  - Context‑aware scoring that tracks how risk evolves across turns.
  - Multi‑task learning to jointly model HIV and mental‑health risk.